1-3. 프롬프트(Prompt)
프롬프트는 사용자와 언어 모델 간의 대화에서 질문이나 요청의 형태로 제시되는 입력문입니다. 이는 모델이 어떤 유형의 응답을 제공할지 결정하는 데 중요한 역할을 합니다.

이 장에서는 랭체인에서 프롬프트를 구성할 때 사용하는 대표적인 도구인 프롬프트 템플릿을 다룹니다.

관련 강의 동영상: 유투브 판다스 스튜디오 (https://youtu.be/wsn7cP3lWwY)

프롬프트 템플릿은 변수를 포함한 문자열 템플릿으로, 동적으로 프롬프트를 생성할 수 있게 해줍니다.
f-string 같은 방식으로 사용할 수 있습니다.


In [1]:
from dotenv import load_dotenv
load_dotenv()

import os
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")


In [5]:
from langchain_core.prompts import PromptTemplate

# 템플릿 정의
template = PromptTemplate.from_template(
    "{topic}에 대해 {length}자 이내로 설명해주세요."
)

# 변수 대입
prompt = template.format(topic="인공지능", length="100")
print(prompt)
# 출력: 인공지능에 대해 100자 이내로 설명해주세요.


인공지능에 대해 100자 이내로 설명해주세요.


In [ ]:
from langchain_core.prompts import PromptTemplate

# 'name'과 'age'라는 두 개의 변수를 사용하는 프롬프트 템플릿을 정의
template_text = "안녕하세요, 제 이름은 {name}이고, 나이는 {age}살입니다."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# 템플릿에 값을 채워서 프롬프트를 완성
filled_prompt = prompt_template.format(name="홍길동", age=30)

print(filled_prompt)

# 문자열 템플릿 결합 (PromptTemplate + PromptTemplate + 문자열)
combined_prompt = (
              prompt_template
              + PromptTemplate.from_template("\n\n아버지를 아버지라 부를 수 없습니다.")
              + "\n\n{language}로 번역해주세요."
)

combined_prompt



안녕하세요, 제 이름은 홍길동이고, 나이는 30살입니다.


PromptTemplate(input_variables=['age', 'language', 'name'], input_types={}, partial_variables={}, template='안녕하세요, 제 이름은 {name}이고, 나이는 {age}살입니다.\n\n아버지를 아버지라 부를 수 없습니다.\n\n{language}로 번역해주세요.')

In [12]:
combined_prompt.format(name="홍길동", age=30, language="영어")


'안녕하세요, 제 이름은 홍길동이고, 나이는 30살입니다.\n\n아버지를 아버지라 부를 수 없습니다.\n\n영어로 번역해주세요.'

1-3-3. 챗 프롬프트 템플릿 (ChatPromptTemplate)

ChatPromptTemplate은 대화형 상황에서 여러 메시지 입력을 기반으로 단일 메시지 응답을 생성하는 데 사용됩니다. 이는 대화형 모델이나 챗봇 개발에 주로 사용됩니다. 입력은 여러 메시지를 원소로 갖는 리스트로 구성되며, 각 메시지는 역할(role)과 내용(content)으로 구성됩니다.



In [2]:
# 2-튜플 형태의 메시지 목록으로 프롬프트 생성 (type, content)

from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "이 시스템은 천문학 질문에 답변할 수 있습니다."),
    ("user", "{user_input}"),
])
messages = chat_prompt.format_messages(user_input="태양계에서 가장 큰 행성은 무엇인가요?")
messages


[SystemMessage(content='이 시스템은 천문학 질문에 답변할 수 있습니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='태양계에서 가장 큰 행성은 무엇인가요?', additional_kwargs={}, response_metadata={})]

In [3]:
from langchain_core.output_parsers import StrOutputParser

chain = chat_prompt | llm | StrOutputParser()

chain.invoke({"user_input": "태양계에서 가장 큰 행성은 무엇인가요?"})

'태양계에서 가장 큰 행성은 목성(Jupiter)입니다. 목성은 지구의 약 1,300배에 해당하는 부피를 가지고 있으며, 그 크기와 질량에서는 태양계의 다른 행성을 모두 압도합니다.'

3. MessagePromptTemplate 활용
다음 예제는 SystemMessagePromptTemplate와 HumanMessagePromptTemplate를 사용하여 천문학 질문에 답변할 수 있는 시스템에 대한 대화형 프롬프트를 생성합니다. ChatPromptTemplate.from_messages 메소드를 통해 시스템 메시지와 사용자 메시지 템플릿을 포함하는 챗 프롬프트를 구성합니다. 이후, chat_prompt.format_messages 메서드를 사용하여 사용자의 질문을 포함한 메시지 리스트를 동적으로 생성합니다.

In [4]:
# MessagePromptTemplate 활용

from langchain_core.prompts import SystemMessagePromptTemplate,  HumanMessagePromptTemplate

chat_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template("이 시스템은 천문학 질문에 답변할 수 있습니다."),
        HumanMessagePromptTemplate.from_template("{user_input}"),
    ]
)

messages = chat_prompt.format_messages(user_input="태양계에서 가장 큰 행성은 무엇인가요?")
messages


[SystemMessage(content='이 시스템은 천문학 질문에 답변할 수 있습니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='태양계에서 가장 큰 행성은 무엇인가요?', additional_kwargs={}, response_metadata={})]

In [5]:
chain = chat_prompt | llm | StrOutputParser()

chain.invoke({"user_input": "태양계에서 가장 큰 행성은 무엇인가요?"})

'태양계에서 가장 큰 행성은 목성(Jupiter)입니다. 목성은 지름이 약 142,984킬로미터에 달하며, 그 질량은 태양계의 다른 모든 행성의 질량을 합한 것보다도 큽니다. 목성은 가스로 이루어진 행성으로, 강력한 자기장과 여러 개의 위성을 가지고 있습니다.'